#### import some library

In [2]:
import easyocr
import keras
from keras import ops
from keras import layers
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model

In [6]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1, dtype='float32', **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

In [7]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim, dtype='float32', **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


#### prepare dataset

In [8]:
# Load the dataset
dataset_path = "D:\Github\colab_ml_model_learning\dataset\data.xlsx"
dataset = pd.read_excel(dataset_path)

texts = dataset["content"].values
labels = dataset["label"].values
vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each tweet

# Tokenize the text data
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

# Pad sequences
padded_sequences = pad_sequences(sequences, maxlen=maxlen)

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels
encoded_labels = label_encoder.fit_transform(labels)

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(padded_sequences, encoded_labels, test_size=0.2, random_state=42)

In [10]:
print(set(y_train))

{0, 1}


In [17]:
embed_dim = 32  # Embedding size for each token
num_heads = 4  # Number of attention heads
ff_dim = 64  # Hidden layer size in feed forward network inside transformer
class_number = 2
num_transformer_layers = 8  # Number of transformer layers

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)

for _ in range(num_transformer_layers):
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)

x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(class_number + 1, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


In [18]:
model.summary()

Model: "functional_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding_3  │ (None, 200, 32)        │       646,400 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_3             │ (None, 200, 32)        │        21,120 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_4             │ (None, 200, 32)        │        21,120 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_5             │ (None, 200, 32)        │        21,120 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_6             │ (None, 200, 32)        │        21,120 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_7             │ (None, 200, 32)        │        21,120 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_8             │ (None, 200, 32)        │        21,120 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_9             │ (None, 200, 32)        │        21,120 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_10            │ (None, 200, 32)        │        21,120 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_3      │ (None, 32)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (None, 20)             │           660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (None, 3)              │            63 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 816,083 (3.11 MB)

 Trainable params: 816,083 (3.11 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=12, validation_data=(x_val, y_val)
)

Epoch 1/12
10/10 ━━━━━━━━━━━━━━━━━━━━ 26s 884ms/step - accuracy: 1.0000 - loss: 0.0162 - val_accuracy: 0.9875 - val_loss: 0.0134
Epoch 2/12
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 761ms/step - accuracy: 1.0000 - loss: 0.0073 - val_accuracy: 1.0000 - val_loss: 8.5729e-04
Epoch 3/12
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 776ms/step - accuracy: 1.0000 - loss: 0.0095 - val_accuracy: 1.0000 - val_loss: 6.0281e-04
Epoch 4/12
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 771ms/step - accuracy: 1.0000 - loss: 0.0035 - val_accuracy: 1.0000 - val_loss: 3.1045e-04
Epoch 5/12
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 771ms/step - accuracy: 1.0000 - loss: 0.0030 - val_accuracy: 1.0000 - val_loss: 1.7699e-04
Epoch 6/12
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 771ms/step - accuracy: 1.0000 - loss: 0.0019 - val_accuracy: 1.0000 - val_loss: 1.2065e-04
Epoch 7/12
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 767ms/step - accuracy: 1.0000 - loss: 0.0022 - val_accuracy: 1.0000 - val_loss: 8.8707e-05
Epoch 8/12
10/10 ━━━━━━━━━━━━━━━━━━━━ 8s 780ms/step - accuracy: 1.0000 - loss: 

#### Save sentiment_model

In [21]:
# Save the model
model.save("./model/activities_tracking_model.h5")

In [9]:
# Load the model
loaded_model = load_model("./model/activities_tracking_model.h5", custom_objects={"TokenAndPositionEmbedding": TokenAndPositionEmbedding, "TransformerBlock": TransformerBlock})

In [1]:
def extract_text(image_path):
    reader = easyocr.Reader(['en'])
    result = reader.readtext(image_path)
    text = " "
    for (bbox, text_s, prob) in result:
        text += text_s
    return text


In [12]:
example_texts = []
test_dir = "D:/Github/colab_ml_model_learning/validation_img"
for img in os.listdir(test_dir):
    example_texts.append(extract_text(os.path.join(test_dir, img)))

print(example_texts)

[' HomeGoogCO vits_fine_tuneTier1 Golc WCO CNN_tensorflcontent overfRUPP-For LecC4C EvaluaticdisposeGocCNN ModelX | +6 +Cyoutube com/watch?v-fbOWYPpTIKEC #6] 0 | =KHYoulubestream gameGgngean5168105$ 120AlIFrom your searchFrom Gosu General TVSOLo CLAUDE2024 Detail Advance Claude +TUTORIALBasic Gold lane TutorialMobi___Gosu General TV453K views3 months ago22.02484i83Shorts5SerueEel Enenaaae862869855RecallRegenAedisUJsondddd -[GENs CONMUNITYarpadenSondgmnDISCORDClick11:03044 FM Sutscibe hereBalmond5 HERO YANGhow oldHealing Tutoria _PALING BESAR_are you_Tier1 Gold Ianer Rogerl?tried MPL Pro players most pickMobile Legends Roger893K views1.9M views2M viewsGosu General TVSubscribe6.9KShareDownload1.82M subscribers3 Hour LongPiano Music &Rain SoundsAudiovisual_18OK views 3 weeks ago #roger #mlbb #mobilelegends1C9 3038 SodyMusicReady for fun?ENG3.14 PMSearchd))Relax and playUS5/3/2024CRBAt', " processor in nlpGoogle SearcMobile Legends: Bang BangChatGPTX | +7youtube com/@MobileLegendsSvSMOBAC 

In [15]:
preprocessed_examples = tokenizer.texts_to_sequences(example_texts)
padded_examples = pad_sequences(preprocessed_examples, maxlen=maxlen)
# Predict using the loaded model
predictions = loaded_model.predict(padded_examples)

# Convert predictions to labels
predicted_labels = label_encoder.inverse_transform(predictions.argmax(axis=1))

print(predicted_labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step
['studying' 'enjoying' 'enjoying' 'studying']
